## Requirement 1: Stochatic Environment

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm 

c:\Users\39327\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\39327\anaconda3\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\Users\39327\anaconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


### Stochastic Pricing Environment

In [2]:
class StochasticPricingEnvironment:
    def __init__(self, model='linear', alpha=None, beta=None):
        self.model = model
        self.alpha = alpha
        self.beta = beta
        np.random.seed(42)


    # given settled price and number customer
    # returns a sample of number of sale and profit
    def round(self, price_t, n_costumer_t):
        # number of sale at time t
        number_sale_t = np.random.binomial(n_costumer_t, StochasticPricingEnvironment.generate_probability(self.model, self.alpha, self.beta, price_t))

        return number_sale_t

    # TODO
    # integrate inside environment the computation of clairvoyant reward

    # takes as input the selling price -> [0,1]
    # return as output the probability of selling -> [0,1]
    @staticmethod
    def generate_probability(model, a, b, price):
        def isNotNone(s,d):
            if s is None:
                return d
            else:
                return s
        
        if model == 'linear':
            alpha = isNotNone(a, 1) # Intercept
            beta = isNotNone(b, -1)  # Slope
            purchase_probabilities = alpha + beta * price
            
        elif model == 'logit':
            alpha = isNotNone(a, 0)
            beta = isNotNone(b, -5)
            purchase_probabilities = 1 / (1 + np.exp(-(alpha + beta * price)))
            
        elif model == 'probit':
            alpha = isNotNone(a, 0)
            beta = isNotNone(b, -5)
            purchase_probabilities = norm.cdf(alpha + beta * price)
            
        elif model == 'KERNEL':
            purchase_probabilities = np.abs(np.sin(2 * np.pi * price) * np.exp(-price * 5) + price * .10) + .1

        return purchase_probabilities


### Pricing Agent : Gaussian Process Agent with RBF kernel and UCB algorithm

In [3]:
# Radial Basis Function Kernel
class RBFGaussianProcess:
    def __init__(self, scale=1, reg=1e-2):
        self.scale = scale 
        self.reg = reg
        self.k_xx_inv = None

    def rbf_kernel_incr_inv(self, B, C, D):
        temp = np.linalg.inv(D - C @ self.k_xx_inv @ B)
        block1 = self.k_xx_inv + self.k_xx_inv @ B @ temp @ C @ self.k_xx_inv
        block2 = - self.k_xx_inv @ B @ temp
        block3 = - temp @ C @ self.k_xx_inv
        block4 = temp
        res1 = np.concatenate((block1, block2), axis=1)
        res2 = np.concatenate((block3, block4), axis=1)
        res = np.concatenate((res1, res2), axis=0)
        return res

    def rbf_kernel(self, a, b):
        a_ = a.reshape(-1, 1)
        b_ = b.reshape(-1, 1)
        output = -1 * np.ones((a_.shape[0], b_.shape[0]))
        for i in range(a_.shape[0]):
            output[i, :] = np.power(a_[i] - b_, 2).ravel()
        return np.exp(-self.scale * output)
    
    def fit(self, x=np.array([]), y=np.array([])):
        x,y = np.array(x),np.array(y)
        if self.k_xx_inv is None:
            self.y = y.reshape(-1,1)
            self.x = x.reshape(-1,1)
            k_xx = self.rbf_kernel(self.x, self.x) + self.reg * np.eye(self.x.shape[0])
            self.k_xx_inv = np.linalg.inv(k_xx)
        else:
            B = self.rbf_kernel(self.x, x)
            self.x = np.vstack((self.x, x))
            self.y = np.vstack((self.y, y))
            self.k_xx_inv = self.rbf_kernel_incr_inv(B, B.T, np.array([1 + self.reg]))

        return self

    def predict(self, x_predict):
        k = self.rbf_kernel(x_predict, self.x)

        mu_hat = k @ self.k_xx_inv @ self.y
        sigma_hat = 1 - np.diag(k @ self.k_xx_inv @ k.T)

        return mu_hat.ravel(), sigma_hat.ravel()

In [31]:
# From the agent's point of view, action set is [0,1]. 
# If the actual actions are outside this set, perform rescaling outside the class.
class GPUCBAgent:
    def __init__(self, T, product_cost, maximum_price=1, scale=0.1, discretization=None):
        self.T = T
        if(discretization is None):
            # discretization prescribed by theory
            epsilon = T**(-0.33)
            discretization = int(1/epsilon)
        self.product_cost = product_cost
        self.arms = np.linspace(product_cost, maximum_price, discretization)
        self.gp = RBFGaussianProcess(scale=scale).fit()
        self.a_t = None
        self.action_hist = np.array([])
        self.reward_hist = np.array([])
        self.mu_t = np.zeros(discretization)
        self.sigma_t = np.zeros(discretization)
        self.gamma = lambda t: np.log(t+1)**2 
        self.beta = lambda t: 1 + 0.5*np.sqrt(2 * (self.gamma(t) + 1 + np.log(T)))
        self.N_pulls = np.zeros(discretization)
        self.t = 0
    
    def pull_arm(self, bidding_cost=0):
        # play every arm at least once
        if self.t < len(self.arms):
            self.a_t = self.t
        else:
            self.mu_t, self.sigma_t = self.gp.predict(self.arms) 
            #    averages of GP  +  estimation uncertainty
            # print(f'mu_t: {self.mu_t}')
            buy_probabilities = self.mu_t + self.beta(self.t) * self.sigma_t
            # print(f'buy probabilities: {buy_probabilities}')
            # print(f'self.arms - bidding_cost: {self.arms - bidding_cost}')
            ucbs = buy_probabilities * (self.arms - bidding_cost)
            # print(f'ucbs: {ucbs}')
            self.a_t = np.argmax(ucbs)
        return self.arms[self.a_t]
    
    def update(self, n_sales_t, n_customers_t, total_cost_t):
        self.N_pulls[self.a_t] += 1
        self.action_hist = np.append(self.action_hist, self.arms[self.a_t])
        self.reward_hist = np.append(self.reward_hist, (self.arms[self.a_t]-total_cost_t)*n_sales_t)
        estimated_probability = n_sales_t / n_customers_t
        if estimated_probability != estimated_probability: # NaN check
            estimated_probability = 0
        self.gp = self.gp.fit(self.arms[self.a_t], estimated_probability)
        self.t += 1

### Bidding Environment

In [5]:
class SecondPriceAuction:
    def __init__(self, ctrs):
        # ctr = click though rate = lambda * q
        self.ctrs = ctrs
        self.n_adv = len(self.ctrs)
    
    def get_winners(self, bids):
        # sort not by bids, but estimated values
        adv_values = self.ctrs*bids
        adv_ranking = np.argsort(adv_values)
        winner = adv_ranking[-1]
        return winner, adv_values
    
    def get_payments_per_click(self, winners, values, bids):
        adv_ranking = np.argsort(values)
        second = adv_ranking[-2]
        payment = values[second]/self.ctrs[winners]
        return payment.round(2)
    
    def round(self, bids):
        # given bids, return winner(s) and the estimated values (product of 3 terms : lambda, q, bid) of winner(s)
        winners, values = self.get_winners(bids) # allocation mechanism!
        payments_per_click = self.get_payments_per_click(winners, values, bids)
        return winners, payments_per_click

In [6]:
class UniformBiddingEnvironment:
    def __init__(self, T, n_advertisers, auction):
        self.T = T
        self.auction = auction
        self.n_advertisers = n_advertisers
        np.random.seed(42)
        self.other_bids = np.random.uniform(0, 1, size = (n_advertisers, T))
        self.m_t = self.other_bids.max(axis=0)

    def round(self, round, bid):
        bids_at_round = np.append(self.other_bids[:, round], bid)
        winners, payments_per_click = self.auction.round(bids=bids_at_round)
        return winners, self.m_t[round]
    
    def get_clairvoyant_truthful(self, B, my_valuation):
        ## I compute my sequence of utilities at every round
        utility = (my_valuation-self.m_t)*(my_valuation>=self.m_t)
        #   if valuation less than max_bid of the round, no need to participate
        #   if higher, utility is the difference

        ## Now I have to find the sequence of m_t summing up to budget B and having the maximum sum of utility
        ## In second price auctions, I can find the sequence **greedily**:
        sorted_round_utility = np.flip(np.argsort(utility)) # sorted rounds, from most profitable to less profitable
        clairvoyant_utilities = np.zeros(self.T)
        clairvoyant_bids= np.zeros(self.T)
        clairvoyant_payments = np.zeros(self.T)
        c = 0
        i = 0
        #    (enough budget) and (examined all rounds)
        while c <= B-1 and i < self.T:
            clairvoyant_bids[sorted_round_utility[i]] = 1  # simply bid 1 because we are in second-price auction
            clairvoyant_utilities[sorted_round_utility[i]] = utility[sorted_round_utility[i]]
            clairvoyant_payments[sorted_round_utility[i]] = self.m_t[sorted_round_utility[i]]
            c += self.m_t[sorted_round_utility[i]]  # cost is incremented by second price of that round
            i+=1
        return clairvoyant_bids, clairvoyant_utilities, clairvoyant_payments

### Bidding Agents

#### Primal-dual algorithm -> Multiplicative Pacing

In [7]:
class MultiplicativePacingAgent:
    def __init__(self, valuation, budget, T, eta):
        self.valuation = valuation
        self.budget = budget
        self.eta = eta # learning rate
        self.T = T
        self.rho = self.budget/self.T
        self.lmbd = 1  # shouldn't this be initialized with 0?
        self.t = 0

    def bid(self):
        if self.budget < 1:
            return 0
        return self.valuation/(self.lmbd+1)
    
    def update(self, f_t, c_t):
        self.lmbd = np.clip(self.lmbd-self.eta*(self.rho-c_t), 
                            a_min=0, a_max=1/self.rho)
        self.budget -= c_t

#### UCB-like algorithm -> simple UCB

In [8]:
class UCBAgent:
    def __init__(self, valuation, budget, T, K=100, range=1):
        self.T = T
        self.valuation = valuation
        self.budget = budget
        self.K = K
        self.range = range
        self.arms = np.linspace(0, valuation, K)
        self.average_rewards = np.zeros(K)
        self.N_pulls = np.zeros(K)
        self.a_t = None
        self.t = 0

    def bid(self):
        # if budget depleted
        if self.budget < 1:
            return 0
        
        # play every arm at least once
        if self.t < self.K:
            self.a_t = self.t
        else:
            # compute UCB for every arms, choose arm with highest UCB
            ucbs = self.average_rewards + self.range*np.sqrt(2*np.log(self.T)/self.N_pulls)
            self.a_t = np.argmax(ucbs)
        return self.arms[self.a_t]

    def update(self, f_t, c_t):
        self.t += 1
        self.N_pulls[self.a_t] += 1
        self.average_rewards[self.a_t] += (f_t - self.average_rewards[self.a_t])/self.N_pulls[self.a_t]
        self.budget -= c_t

#### UCB-like algorithm -> UCB with negative feedback when not winning (lost opportunity)
lost opportunity computed as average utility when winning

In [9]:
class UCBNFAgent:
    def __init__(self, valuation, budget, T, K=None, range=1):
        self.T = T
        self.valuation = valuation
        self.budget = budget
        if K is None:
            # prescribed by theory
            epsilon = T**(-0.33)
            K = int(1/epsilon)
        self.K = K
        self.range = range
        self.arms = np.linspace(0, valuation, K)
        self.average_rewards = np.zeros(K)
        self.N_pulls = np.zeros(K)
        self.winning_total_reward = 0
        self.won_time = 0
        self.a_t = None
        self.t = 0

    def bid(self):
        # if budget depleted
        if self.budget < 1:
            return 0
        
        # play every arm at least once
        if self.t < self.K:
            self.a_t = self.t
        else:
            # compute UCB for every arms, choose arm with highest UCB
            ucbs = self.average_rewards + self.range*np.sqrt(2*np.log(self.T)/self.N_pulls)
            self.a_t = np.argmax(ucbs)
            # if(self.t<50):
            #     print(f'round {self.t} : {ucbs}\n bidded {self.arms[self.a_t]} with ucb={ucbs[self.a_t]}')
        return self.arms[self.a_t]

    def update(self, f_t, c_t):
        self.N_pulls[self.a_t] += 1
        
        if(f_t>0):
            self.won_time += 1
            self.winning_total_reward += f_t
        elif(self.t >= self.K):
            f_t = - (self.winning_total_reward / self.won_time)
    
        self.average_rewards[self.a_t] += (f_t - self.average_rewards[self.a_t])/self.N_pulls[self.a_t]
        self.budget -= c_t
        self.t += 1

### Simulation

In [42]:
def MinMaxScaler(original, min_x, max_x):
    return (original - min_x) / (max_x - min_x)

def MinMaxRescaler(scaled, min_x, max_x):
    return min_x + (max_x - min_x) * scaled


np.random.seed(2)
# common environment
TotalDays = 365

# product 
min_price, max_price = 10, 20
production_cost = 10
price_discretization = 100
mean_payed_bid_per_user = 0     # [0, max_price - production_cost]

# Pricing Environment, deals with purchase probability
pricing_env = StochasticPricingEnvironment(model='linear', alpha=0.9, beta=-0.8)
# Pricing Agent
pricing_agent = GPUCBAgent(TotalDays, production_cost, max_price, scale=1)

prices = np.linspace(min_price, max_price, price_discretization)
n_customers_t = 0        # to be determined every day after bidding



# Foreach day
for day in range(TotalDays):

    # pricing agent set daily price : [normal scale] (from product_cost to max_price)
    p_t = pricing_agent.pull_arm(bidding_cost=mean_payed_bid_per_user)


    # Bidding Environment, deals with number of auctions hold every day, number and bidding strategy of opponents
    n_users = 10000
    n_competitors = 3
    ctrs = np.ones(n_competitors+1)
    my_valuation_t = MinMaxScaler(p_t, min_price, max_price) # equals to daily_set_price (which took into account the production cost)
    B = 5000
    auction = SecondPriceAuction(ctrs)
    bidding_env = UniformBiddingEnvironment(n_users, n_competitors, auction)


    # Bidding Agent
    # valuation and budget can be modified on fly everyday
    eta = 1/np.sqrt(n_users) # from theory
    agentMP = MultiplicativePacingAgent(valuation=my_valuation_t,
                                    budget=B,
                                    T=n_users, 
                                    eta=eta)
    agentUCB = UCBAgent(my_valuation_t, B, n_users, range=1)
    agentNF = UCBNFAgent(my_valuation_t, B, n_users, range=0.5)
    agentNF1 = UCBNFAgent(my_valuation_t, B, n_users, range=0.1)    # range works as the hyperparameter in this problem
    # max_user= 100
    # bidding_agent = ContinuousUpdatedRhoMPAgent(valuation=my_value,
    #                                   budget=B,
    #                                   T=max_user, 
    #                                   eta=1/math.sqrt(max_user)) # from theory

    bidding_agents = {
        "Multiplicative Pacing": agentMP,
        # "Plain UCB": agentUCB,
        # "UCB with negative feedback": agentNF,
        # "UCBNF with range 0.1" : agentNF1
    }

    utilities = np.zeros((len(bidding_agents), n_users))
    my_bids = np.zeros((len(bidding_agents), n_users))
    my_payments = np.zeros((len(bidding_agents), n_users))
    total_wins = np.zeros((len(bidding_agents)))

    np.random.seed(day)
    for u in range(n_users):
        # interaction
        for idx, (name, agent) in enumerate(bidding_agents.items()):
            my_bid = agent.bid()
            winners, m_t = bidding_env.round(u, my_bid)
            my_win = int(winners==n_competitors)
            f_t, c_t = (my_valuation_t-m_t)*my_win, m_t*my_win
            agent.update(f_t, c_t)
            utilities[idx,u] = f_t
            my_bids[idx,u] = my_bid
            my_payments[idx,u] = c_t
            total_wins[idx] += my_win
    # print(f'Total # of Wins: {total_wins}')
    n_customers_t = total_wins[0]
    mean_payed_bid_per_user = np.sum(my_payments[0,:]) / total_wins[0]      # [0,1]
    mean_payed_bid_per_user = mean_payed_bid_per_user*(max_price-min_price) # [0, max_price - production_cost] normal scale

    number_sale_t = pricing_env.round(MinMaxScaler(p_t, min_price, max_price), n_customers_t)
    
    # update pricing agent
    pricing_agent.update(number_sale_t, n_customers_t, production_cost+mean_payed_bid_per_user)


    # logging
    print(f'At day {day} :\tprice_t = {p_t:.2f}, my valuation for bidding = {my_valuation_t:.2f} \
          \n\twe won {int(n_customers_t)}/{n_users} auctions, \
          with a mean bid of {mean_payed_bid_per_user/(max_price-min_price):.2f}({mean_payed_bid_per_user:.2f}) \
          \n\twe sold {number_sale_t}/{int(n_customers_t)} products, for a final profit of {number_sale_t*(p_t-production_cost-mean_payed_bid_per_user)}')
    
    # print(f'mean_payed_bid_per_user: {mean_payed_bid_per_user}')
    # print(f'mu_t: {pricing_agent.mu_t}')
    # buy_probabilities = pricing_agent.mu_t + pricing_agent.beta(pricing_agent.t) * pricing_agent.sigma_t
    # print(f'buy probabilities: {buy_probabilities}')
    # print(f'self.arms - bidding_cost: {pricing_agent.arms - mean_payed_bid_per_user}')
    # ucbs = buy_probabilities * (pricing_agent.arms - mean_payed_bid_per_user)
    # print(f'ucbs: {ucbs}')
    # print()

    

C:\Users\39327\AppData\Local\Temp\ipykernel_19152\913458031.py:88: RuntimeWarning: invalid value encountered in double_scalars
  mean_payed_bid_per_user = np.sum(my_payments[0,:]) / total_wins[0]      # [0,1]
C:\Users\39327\AppData\Local\Temp\ipykernel_19152\2477525892.py:43: RuntimeWarning: invalid value encountered in double_scalars
  estimated_probability = n_sales_t / n_customers_t


At day 0 :	price_t = 10.00, my valuation for bidding = 0.00           
	we won 0/10000 auctions,           with a mean bid of nan(nan)           
	we sold 0/0 products, for a final profit of nan
At day 1 :	price_t = 11.67, my valuation for bidding = 0.17           
	we won 41/10000 auctions,           with a mean bid of 0.13(1.30)           
	we sold 32/41 products, for a final profit of 11.787775275326524
At day 2 :	price_t = 13.33, my valuation for bidding = 0.33           
	we won 354/10000 auctions,           with a mean bid of 0.25(2.52)           
	we sold 212/354 products, for a final profit of 173.2048531985783
At day 3 :	price_t = 15.00, my valuation for bidding = 0.50           
	we won 1195/10000 auctions,           with a mean bid of 0.38(3.75)           
	we sold 601/1195 products, for a final profit of 749.925878785416
At day 4 :	price_t = 16.67, my valuation for bidding = 0.67           
	we won 2930/10000 auctions,           with a mean bid of 0.50(5.02)           
	we 

KeyboardInterrupt: 

In [38]:
print(f'mean_payed_bid_per_user: {mean_payed_bid_per_user}')
print(f'mu_t: {pricing_agent.mu_t}')
buy_probabilities = pricing_agent.mu_t + pricing_agent.beta(pricing_agent.t) * pricing_agent.sigma_t
print(f'buy probabilities: {buy_probabilities}')
print(f'self.arms - bidding_cost: {pricing_agent.arms - mean_payed_bid_per_user}')
ucbs = buy_probabilities * (pricing_agent.arms - mean_payed_bid_per_user)
print(f'ucbs: {ucbs}')



mean_payed_bid_per_user: 1.2982986893127122
mu_t: [4.57420074e-04 7.72427820e-01 5.93666493e-01 4.89451380e-01
 3.77168658e-01 2.24529757e-01 1.03035385e-01]
buy probabilities: [0.03770069 0.77618575 0.63090833 0.50816566 0.4144105  0.26177162
 0.14027867]
self.arms - bidding_cost: [ 8.70170131 10.36836798 12.03503464 13.70170131 15.36836798 17.03503464
 18.70170131]
ucbs: [0.32806018 8.04777946 7.59300357 6.96273412 6.36881313 4.45928855
 2.62344983]
